# Assignment 3 (Concise, without reading data from external source)



$$ \sum_{i=A,...G,j=1,..,4} (prod\_cost_{i,j}+trans\_cost_{i,j})x_{i,j}\\
s.t. \\
Demand: \sum_{i=A,...,G}x_{ij} \ge Demand_i,  j=1,2,3,4\\
Capacity: \sum_{j=1,...,4}proc\_time_{i,j}x_{i,j}\le i=A,...,G \\
A1: x_{A,1}=0 \\
D1: x_{D,1}=0 \\
$$

## Import PuLP modeler functions

In [1]:
from pulp import *
#import pandas as pd
#import numpy as np

## Question 1

In [139]:
probA=LpProblem("Problem A",LpMinimize)

factory=['A','B','C','D','E','F','G']

yarn = ['1','2','3','4']

# Creates a list of production cost
prod_cost =[ [0,13.00, 10.65, 9.60],\
            [ 17.40,14.10,11.20,9.45],\
            [ 17.40,14.22,11.00,9.50],\
            [0,14.30,11.25,9.60],\
            [ 17.50, 13.80,11.40,9.60],\
            [ 18.25, 13.90,11.40, 8.90],\
            [ 19.75,13.90,10.75,9.40 ]]

prod_cost= makeDict([factory,yarn],prod_cost)

# Creates a list of transportation cost
trans_cost = [
    [ 0.30,0.30,0.45, 0.45],\
    [ 0.40,0.40,0.60,0.60],\
    [ 0.80,0.80, 1.20,1.20],\
    [ 0.70,0.70,1.05,1.05],\
    [ 0.70, 0.70, 1.05, 1.05],\
    [ 0,0,0,0],\
    [ 0.50, 0.50, 0.75, 0.75]]

trans_cost= makeDict([factory,yarn],trans_cost)

proc_time=[
    [0, 0.400,0.375,0.250],\
    [0.700,0.500,0.350,0.250],\
    [0.675,0.450,0.400,0.250],\
    [0,0.450,0.350,0.200],\
    [0.650,0.450,0.400,0.250],\
    [0.625,0.500,0.425,0.425],\
    [0.700,0.450,0.350,0.400],\
]
proc_time= makeDict([factory,yarn],proc_time)
capacity = {
    "A":2500,
"B":3000,
"C":2500,
"D":2600,
"E":2500,
"F":38000,
"G":2500}
demand={
    "1":25000,
    "2":26000,
    "3":28000,
    "4":28000
}

production_vars = LpVariable.dicts("x", (factory,yarn),lowBound=0, cat='Continuous')

### Question 1: Add objective function and constraints
- Use lpSum() amd += operator to construct objective function and constraintsumn based modelling: relates to the variable’s existence in the objective function and constraints

In [140]:
#objective function
probA+=lpSum([production_vars[i][j]*(prod_cost[i][j]+trans_cost[i][j]) for i in factory for j in yarn]),"total cost"
# demand constraint
for j in yarn:
    probA += lpSum([production_vars[i][j] for i in factory ]) >= demand[j],"demand_%s"%j

# capacity constraint    
for i in factory:
    probA += lpSum([proc_time[i][j]*production_vars[i][j] for j in yarn]) <= capacity[i],"capacity_%s"%i

# x_1A and x_1D constraint
probA += production_vars["A"]["1"]==0,"xA1=0"
probA += production_vars["D"]["1"]==0,"xD1=0"

### Question 1: Run solver

- use name.solve(solver=None), where name is the LP problem variable defined by LpProblem
- Solve the given Lp problem. 
- This function changes the problem to make it suitable for solving then calls the solver.actualSolve method to find the solution. 
- solver – Optional: the specific solver to be used, defaults to the default solver.

In [141]:
probA.writeLP("FilatoiA.lp")
probA.solve()
print("Status:",LpStatus[probA.status])

Status: Optimal


### Question 1: Print the optiomal solution

In [142]:
for v in probA.variables():
    print(v.name, "=", v.varValue,"\tReduced Cost =", v.dj)
print("Total cost =", value(probA.objective))
original_cost=value(probA.objective)

x_A_1 = 0.0 	Reduced Cost = 0.0
x_A_2 = 6250.0 	Reduced Cost = 0.0
x_A_3 = 0.0 	Reduced Cost = 0.35514706
x_A_4 = 0.0 	Reduced Cost = 1.2867647
x_B_1 = 4285.7143 	Reduced Cost = 0.0
x_B_2 = 0.0 	Reduced Cost = 0.80798319
x_B_3 = 0.0 	Reduced Cost = 0.38676471
x_B_4 = 0.0 	Reduced Cost = 0.88340336
x_C_1 = 3703.7037 	Reduced Cost = 0.0
x_C_2 = 0.0 	Reduced Cost = 0.97686275
x_C_3 = 0.0 	Reduced Cost = 0.71394336
x_C_4 = 0.0 	Reduced Cost = 1.4087146
x_D_1 = 0.0 	Reduced Cost = 0.0
x_D_2 = 0.0 	Reduced Cost = 0.041176471
x_D_3 = 2040.1255 	Reduced Cost = 0.0
x_D_4 = 0.0 	Reduced Cost = 0.85
x_E_1 = 3846.1538 	Reduced Cost = -2.220446e-16
x_E_2 = 0.0 	Reduced Cost = 0.49208145
x_E_3 = 0.0 	Reduced Cost = 0.99524887
x_E_4 = 0.0 	Reduced Cost = 1.3782805
x_F_1 = 13164.428 	Reduced Cost = 0.0
x_F_2 = 19750.0 	Reduced Cost = 0.0
x_F_3 = 18817.017 	Reduced Cost = 7.7715612e-16
x_F_4 = 28000.0 	Reduced Cost = -9.9920072e-16
x_G_1 = 0.0 	Reduced Cost = 2.2764706
x_G_2 = 0.0 	Reduced Cost = 0.469

In [143]:
print("\nSensitivity Analysis\nConstraint\t\tShadow Price\tSlack")
for name, c in list(probA.constraints.items()):
    print(name, ":", c, "\t", c.pi, "\t\t", c.slack)


Sensitivity Analysis
Constraint		Shadow Price	Slack
demand_1 : x_A_1 + x_B_1 + x_C_1 + x_D_1 + x_E_1 + x_F_1 + x_G_1 >= 25000 	 19.573529 		 -0.0
demand_2 : x_A_2 + x_B_2 + x_C_2 + x_D_2 + x_E_2 + x_F_2 + x_G_2 >= 26000 	 14.958824 		 -0.0
demand_3 : x_A_3 + x_B_3 + x_C_3 + x_D_3 + x_E_3 + x_F_3 + x_G_3 >= 28000 	 12.3 		 -0.0
demand_4 : x_A_4 + x_B_4 + x_C_4 + x_D_4 + x_E_4 + x_F_4 + x_G_4 >= 28000 	 9.8 		 -0.0
capacity_A : 0.4*x_A_2 + 0.375*x_A_3 + 0.25*x_A_4 <= 2500.0 	 -4.1470588 		 -0.0
capacity_B : 0.7*x_B_1 + 0.5*x_B_2 + 0.35*x_B_3 + 0.25*x_B_4 <= 3000.0 	 -2.5336134 		 -0.0
capacity_C : 0.675*x_C_1 + 0.45*x_C_2 + 0.4*x_C_3 + 0.25*x_C_4 <= 2500.0 	 -2.0348584 		 -0.0
capacity_D : 0.45*x_D_2 + 0.35*x_D_3 + 0.2*x_D_4 <= 2600.0 	 0.0 		 1885.9560900000001
capacity_E : 0.65*x_E_1 + 0.45*x_E_2 + 0.4*x_E_3 + 0.25*x_E_4 <= 2500.0 	 -2.1131222 		 -0.0
capacity_F : 0.625*x_F_1 + 0.5*x_F_2 + 0.425*x_F_3 + 0.425*x_F_4 <= 38000.0 	 -2.1176471 		 -0.0
capacity_G : 0.7*x_G_1 + 0.45*x_G_2 + 

## Question 2

In [122]:
# change capacity for Filatoi from 38000 to 38000+600=38600
probB=LpProblem("Problem B",LpMinimize)
capacity["F"]=38600

In [79]:
capacity

{'A': 2500, 'B': 3000, 'C': 2500, 'D': 2600, 'E': 2500, 'F': 38600, 'G': 2500}

In [80]:
#objective function
probB+=lpSum([production_vars[i][j]*(prod_cost[i][j]+trans_cost[i][j]) for i in factory for j in yarn]),"total cost"
# demand constraint
for j in yarn:
    probB += lpSum([production_vars[i][j] for i in factory ]) >= demand[j],"demand_%s"%j

# capacity constraint    
for i in factory:
    probB += lpSum([proc_time[i][j]*production_vars[i][j] for j in yarn]) <= capacity[i],"capacity_%s"%i

# x_1A and x_1D constraint
probB += production_vars["A"]["1"]==0,"xA1=0"
probB += production_vars["D"]["1"]==0,"xD1=0"
#solve LP
probB.writeLP("Filatoi2b.lp")
probB.solve()
print("Status:",LpStatus[probA.status])
#print optimal solution
for v in probB.variables():
    print(v.name, "=", v.varValue,"\tReduced Cost =", v.dj)
print("Total cost =", value(probB.objective))
probB_cost = value(probB.objective)
print("new cost-original cost =", probB_cost-original_cost)

Status: Optimal
x_A_1 = 0.0 	Reduced Cost = 0.0
x_A_2 = 6250.0 	Reduced Cost = 0.0
x_A_3 = 0.0 	Reduced Cost = 0.35514706
x_A_4 = 0.0 	Reduced Cost = 1.2867647
x_B_1 = 4285.7143 	Reduced Cost = 0.0
x_B_2 = 0.0 	Reduced Cost = 0.80798319
x_B_3 = 0.0 	Reduced Cost = 0.38676471
x_B_4 = 0.0 	Reduced Cost = 0.88340336
x_C_1 = 3703.7037 	Reduced Cost = 0.0
x_C_2 = 0.0 	Reduced Cost = 0.97686275
x_C_3 = 0.0 	Reduced Cost = 0.71394336
x_C_4 = 0.0 	Reduced Cost = 1.4087146
x_D_1 = 0.0 	Reduced Cost = 0.0
x_D_2 = 0.0 	Reduced Cost = 0.041176471
x_D_3 = 628.36075 	Reduced Cost = 0.0
x_D_4 = 0.0 	Reduced Cost = 0.85
x_E_1 = 3846.1538 	Reduced Cost = -2.220446e-16
x_E_2 = 0.0 	Reduced Cost = 0.49208145
x_E_3 = 0.0 	Reduced Cost = 0.99524887
x_E_4 = 0.0 	Reduced Cost = 1.3782805
x_F_1 = 13164.428 	Reduced Cost = 0.0
x_F_2 = 19750.0 	Reduced Cost = 0.0
x_F_3 = 20228.782 	Reduced Cost = 7.7715612e-16
x_F_4 = 28000.0 	Reduced Cost = -9.9920072e-16
x_G_1 = 0.0 	Reduced Cost = 2.2764706
x_G_2 = 0.0 	Redu

## Question 3.1 (lower Filatoi's prod_cost by 5%)

In [144]:
probC=LpProblem("Problem C",LpMinimize)
# reset capacity and prod_cost
capacity = {
    "A":2500,
"B":3000,
"C":2500,
"D":2600,
"E":2500,
"F":38000,
"G":2500}
prod_cost_l =[ [0,13.00, 10.65, 9.60],\
            [ 17.40,14.10,11.20,9.45],\
            [ 17.40,14.22,11.00,9.50],\
            [0,14.30,11.25,9.60],\
            [ 17.50, 13.80,11.40,9.60],\
            [ 18.25, 13.90,11.40, 8.90],\
            [ 19.75,13.90,10.75,9.40 ]]

prod_cost_l= makeDict([factory,yarn],prod_cost_l)
#adjust prod_cost for Filatoi
adjust_factor=0.05
for j in yarn:
    #print("original prod_cost_l[F][%s]"%j, "=", prod_cost_l["F"][j])
    prod_cost_l["F"][j] = prod_cost_l["F"][j]*(1-adjust_factor)
    #print("new prod_cost[F][%s]"%j, "=", prod_cost_l["F"][j])

#define new decision variable
production_vars_l = LpVariable.dicts("x", (factory,yarn),lowBound=0, cat='Continuous')

In [145]:
#objective function
probC+=lpSum([production_vars_l[i][j]*(prod_cost_l[i][j]+trans_cost[i][j]) for i in factory for j in yarn]),"total cost"
# demand constraint
for j in yarn:
    probC += lpSum([production_vars_l[i][j] for i in factory ]) >= demand[j],"demand_%s"%j

# capacity constraint    
for i in factory:
    probC += lpSum([proc_time[i][j]*production_vars_l[i][j] for j in yarn]) <= capacity[i],"capacity_%s"%i

# x_1A and x_1D constraint
probC += production_vars_l["A"]["1"]==0,"xA1=0"
probC += production_vars_l["D"]["1"]==0,"xD1=0"
#solve LP
probC.writeLP("Filatoi2C.lp")
probC.solve()
print("Status:",LpStatus[probC.status])
#print optimal solution
for v in probC.variables():
    print(v.name, "=", v.varValue,"\tReduced Cost =", v.dj)
print("Total cost =", value(probC.objective))

for u,v in zip(probA.variables(),probC.variables()):
    print("Difference in", v.name,"=",u.varValue-v.varValue)

#for i in factory:
#    for j in yarn:
#        print(value(production_vars[i][j])-value(production_vars_l[i][j]))

Status: Optimal
x_A_1 = 0.0 	Reduced Cost = 0.0
x_A_2 = 6250.0 	Reduced Cost = 0.0
x_A_3 = 0.0 	Reduced Cost = 0.33226103
x_A_4 = 0.0 	Reduced Cost = 1.1465074
x_B_1 = 4285.7143 	Reduced Cost = 0.0
x_B_2 = 0.0 	Reduced Cost = 0.77934874
x_B_3 = 0.0 	Reduced Cost = 0.34963235
x_B_4 = 0.0 	Reduced Cost = 0.73188025
x_C_1 = 3703.7037 	Reduced Cost = 0.0
x_C_2 = 0.0 	Reduced Cost = 0.95176471
x_C_3 = 0.0 	Reduced Cost = 0.66993464
x_C_4 = 0.0 	Reduced Cost = 1.2562092
x_D_1 = 0.0 	Reduced Cost = 0.0
x_D_2 = 0.0 	Reduced Cost = 0.065588235
x_D_3 = 2040.1255 	Reduced Cost = 0.0
x_D_4 = 0.0 	Reduced Cost = 0.725
x_E_1 = 3846.1538 	Reduced Cost = -2.220446e-16
x_E_2 = 0.0 	Reduced Cost = 0.46507919
x_E_3 = 0.0 	Reduced Cost = 0.94954751
x_E_4 = 0.0 	Reduced Cost = 1.2247172
x_F_1 = 13164.428 	Reduced Cost = 0.0
x_F_2 = 19750.0 	Reduced Cost = -8.8817842e-16
x_F_3 = 18817.017 	Reduced Cost = 1.110223e-15
x_F_4 = 28000.0 	Reduced Cost = -6.6613381e-16
x_G_1 = 0.0 	Reduced Cost = 2.3507353
x_G_2 